# FirecREST Tutorial - Part 3: Data Transfer


Set your credentials like before and run the cell. You should get status code `200`.

#### Cell 3A

In [ ]:
import json
import requests
import time

# Set you token here
TOKEN='<token>'
FIRECREST_IP = 'http://148.187.98.88:8000'

# This function is useful only to print the response in a nicer way
def handle_response(response):
    print("\nResponse status code:")
    print(response.status_code)
    print("\nResponse headers:")
    print(json.dumps(dict(response.headers), indent=4))
    print("\nResponse json:")
    try:
        print(json.dumps(response.json(), indent=4))
    except json.JSONDecodeError:
        print("-")
        
response = requests.get(
    url=f'{FIRECREST_IP}/status/systems',
    headers={'Authorization': f'Bearer {TOKEN}'}
)

handle_response(response)

## Upload a bigger file through the Storage microservice

For uploading small files the blocking call that we used in a previous section is enough. When the file we want to upload to a machine’s filesystem is bigger than 5MB, we have to use the Storage microservice.

In order for you to do this, you won't upload the file directly through FirecREST but to a staging area. This staging area is the Object Storage and you can get more information [here](https://user.cscs.ch/storage/object_storage/).

This task will be split into more steps but it will correspond to one FirecREST task, so we have to keep track of one task ID.
The steps are:

1. the user asks FirecREST for a link to the staging area
2. the user uploads the file to the staging area
3. FirecREST will poll the staging area and, as soon as the file is uploaded it will transfer it to the filesystem

The FirecREST task will have the following status codes in different stages of the upload:

Status Code | Description
:---: | :---
110 | Waiting for Form URL from Object Storage to be retrieved
111 | Form URL from Object Storage received
112 | Object Storage confirms that upload to Object Storage has finished
113 | Download from Object Storage to server has started
114 | Download from Object Storage to server has finished
115 | Download from Object Storage error

The first step is to send a `POST` request to FirecREST, to the `/storage/xfer-external/upload` endpoint. Besides the `Authorization` token, we have to include the local path of the file we are going to upload (`sourcePath`) and the target location of the transfer (`targetPath`). Both `sourcePath` and `targetPath` are form data parameters.

> A small tip before running the next cell. You can click on the output of the cell to minimize it and double click to hide it completely.

#### Cell 3B

In [ ]:
targetPath = '/home/llama'
sourcePath = 'files/firecrest_input_file.txt'

response = requests.post(
    url=f'{FIRECREST_IP}/storage/xfer-external/upload',
    headers={'Authorization': f'Bearer {TOKEN}'},
    data={'targetPath': targetPath,
          'sourcePath': sourcePath}
)

handle_response(response)

# response.ok will be True if no error occured
if response.ok:
    taskid = response.json()['task_id']
    
    print(f"\n{50*'.'}")
    time.sleep(1)
    
    response = requests.get(
        url=f'{FIRECREST_IP}/tasks/{taskid}',
        headers={'Authorization': f'Bearer {TOKEN}'}
    )

    handle_response(response)

If the call was successful, the output will be very long. The parts that we care about are the `status` of the FirecREST task, which should be `Form URL from Object Storage received` and the command that FirecREST provides.

In python json objects are just dictionaries, so we can easily isolate the fields we are interested in.

#### Cell 3C

In [ ]:
print(f"Task ID: {response.json()['task']['hash_id']}\n"
      f"Task status code: {response.json()['task']['status']}\n"
      f"Task status description: {response.json()['task']['description']}\n")

# You can isolate the "command" field, that holds the useful information
print(response.json()['task']['data']['msg']['command'])

Copy the command after the exclamation mark in the cell below to execute it.

#### Cell 3D

In [ ]:
# Add the curl command after the exclamation mark and run the cell
! curl ...

### Exercise:

Let's check again on the FirecREST task, we have done it already many times. Make a `GET` request to `/tasks/{taskid}` and get the status of the task. What is the status of the task now?

<a id='get-status-cell'></a>
#### Cell 3E

In [ ]:
# Try to fill the call yourself and run it here.
# If you run the next cell you will get the answer.


#### Cell 3F

In [ ]:
%cat solutions/get_status_id.py


### Exercise: Download the output through the Storage microservice

Just like we did with the `/storage/xfer-external/download` call, the user will not directly download the file from FirecREST but through a staging area.

The workflow of this task is:

1. The user has to make a request to FirecREST to download the file from `sourcePath`, this should be the absolute path in the `cluster` filesystem. This requires at least two calls from the user:
  1. Make a request to FirecREST to create the task of external uploading.
  2. Poll FirecREST the task until the file has been transfered to the staging area. When it is ready FirecREST will reply with the URL from which we will download it.
2. After receiving the URL you have to download the file with a simple `wget` from the link that will be provided by FirecREST.

The FirecREST task will have the following status codes in different stages of the download:

Status Code | Description
:---: | :---
116 | Started upload from filesystem to Object Storage
117 | Upload from filesystem to Object Storage has finished succesfully
118 | Upload from filesystem to Object Storage has finished with errors

#### Step 1A: Make the FirecREST request

In order to fill the request keep in mind that:

- It is a `POST` request.
- The endpoint is `/storage/xfer-external/download`.
- You should pass in the header arguments the `Authorization` token.
- The last argument is `sourcePath` and it a form parameter.

#### Cell 3G

In [ ]:
sourcePath = '/home/llama/res.txt'

response = requests. # Try to fill the request yourself

handle_response(response)

You can get the answer by running the next cell.

#### Cell 3H

In [ ]:
%cat solutions/external_download.py


#### Step 1B: Based on the reply, check for the status of the task.

You have already done this part in [this](#get-status-cell) exercise. Copy it with the correct `taskid`.

#### Cell 3I

In [ ]:
# Make a request to FirecREST to get the status of the task you just created.


Through python we can isolate the field of the response that we are interested in: 

#### Cell 3J

In [ ]:
print(response.json()['task']['data'])

#### Cell 3K

In [ ]:
# Copy the link and finish the download in bash
!wget -O res.txt "..."


#### Cell 3L

In [ ]:
!cat res.txt

#### Cell 3M

In [ ]:
!sha1sum firecrest_input_file.txt

If everything went right you should be getting the same result. Now feel free to create more python cells and experiment with more FirecREST calls.